In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("/home/ray__/ssd/BERT/")
sys.path.append("/home/ray__/CS/org/etherlabs/ai-engine/pkg/")
sys.path.append("../")
from gpt_feat_utils import GPT_Inference

gpt_model = GPT_Inference("/home/ray__/ssd/BERT/models/se/epoch3/", device="cpu")

with open('../topic_testing/cullen_test.json','rb') as f:
    request = json.load(f)
    if isinstance(request, str):
        request = json.loads(request)


ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [3]:
import pickle
import numpy as np
import networkx as nx
# # S.E
# ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/se/se_entity_feats_v3.pkl", "rb"))
# com_map = pickle.load(open("/home/ray__/ssd/minds/se/com_map_se_h_v3", "rb"))
# ent_graph = nx.read_gpickle("/home/ray__/ssd/minds/se/se_ent_graph_wcosine_pruned_v3.gpkl")

# A.I
ent_fv_full = pickle.load(open("/home/ray__/ssd/minds/ai/ai_entity.pkl", "rb"))
com_map = pickle.load(open("/home/ray__/ssd/minds/ai/com_map_ai.pkl", "rb"))
ent_graph = nx.read_gpickle("/home/ray__/ssd/minds/ai/ai_pruned_entity_wfv.gpkl")


common_entities = ent_fv_full.keys() & com_map.keys()
ent_fv = {}
for ent in common_entities:
    ent_fv[ent] = ent_fv_full[ent]

In [7]:
import networkx as nx

clusters_unf = {}
clusters_fv = {}
clusters = {}
for ent, cls in com_map.items():
    if cls not in clusters_unf.keys():
        clusters_unf[cls] = []
        clusters_fv[cls] = []
    clusters_unf[cls].append(ent)
    clusters_fv[cls].append(ent_fv[ent])

    
for cls, cluster in clusters_unf.items():
    if len(cluster)<=5:
        clusters_fv.pop(cls)
        continue
    else:
        clusters[cls] = cluster

In [8]:
from networkx import pagerank 
influential_nodes = {}
for cls, cluster in clusters.items():
    G = ent_graph.subgraph(cluster)
    influential_nodes[cls] = sorted(pagerank(G).items(), key=lambda kv:kv[1], reverse=True)[0][0]

In [9]:
from scipy.spatial.distance import cosine
G = nx.Graph()
G.add_nodes_from(influential_nodes.keys())
for index1, nodea in enumerate(G.nodes()):
    for index2, nodeb in enumerate(G.nodes()):
        if index2>=index1:
            G.add_edge(nodea, nodeb, weight = 1-cosine(ent_fv[influential_nodes[nodea]], ent_fv[influential_nodes[nodeb]]))

In [10]:
pg = sorted(pagerank(G, weight="weight").items(), key=lambda kv:kv[1], reverse=True)

In [11]:
pg

[(6, 0.01825915640297551),
 (1, 0.01778679267124613),
 (11, 0.017749121841297823),
 (59, 0.017619002388781686),
 (12, 0.016972451874929084),
 (60, 0.016840463301358168),
 (66, 0.016785855127979168),
 (141, 0.016580851951487105),
 (35, 0.016532323294926893),
 (63, 0.016275593436982014),
 (26, 0.016065250766392113),
 (288, 0.016048507295442833),
 (85, 0.016027398534094156),
 (17, 0.015942689359247728),
 (43, 0.01594141757858696),
 (64, 0.01591324440759251),
 (8, 0.015886579124495665),
 (78, 0.015821791182128273),
 (50, 0.015784614312266434),
 (10, 0.015717962909807932),
 (14, 0.01569415842748389),
 (15, 0.01568627636526988),
 (40, 0.015666221532815463),
 (23, 0.015635878462214006),
 (454, 0.01560825652135988),
 (90, 0.015537285425924849),
 (1253, 0.01545373241911937),
 (39, 0.015392866774327623),
 (49, 0.015339663928921504),
 (201, 0.015334249413197085),
 (57, 0.015302373088647769),
 (450, 0.015203519223308865),
 (33, 0.015170429408500524),
 (13, 0.015145086833742867),
 (129, 0.015144898

In [15]:
for pos, cls in enumerate(pg):
    print ("\n\nRank: ", pos, "\n\n")
    print (*clusters[cls[0]], sep=", ")



Rank:  0 


University, Bristol, Google AI, Stanford, Geoffrey Hinton, Jeff Dean, Ilya Sutskever, Arts, Richards, NIPS, Yann LeCun, Andrew Ng, Google Brain, LeCuns, Rachel, Jeremy, Africa, South Africa, Udemy, Nairobi, Kenya, Moustapha Cisse, Deep Learning Indaba, MIT, Joanna, Princeton University, Stanford University, Ng, Carnegie Mellon University, Switzerland, GOFAI, ICLR, Coursera, David Silver, Deepmind, Kyunghun Cho, NYU, African, Lab, Sebastian Ruder, Design, George Washington University, Chris, Harvard, GE, NECSI, Shen, College, Hinton, Kingma, Andrew Ngs, Gary Marcus, LeCun, Yoshua Bengio, Alex Krizhevsky, Peter Norvig, Copenhagen, Denmark, ICML, Udacity, Hintons, Personal Robots, Media Lab, CVPR, ACM, Innovation, Microsoft Research, Geoff Hinton, Chen, Princeton, Learning Representations, School, Bengios, Jeremy Howard, Symbolic AI, ACL, Business Administration, MOOC, Conference, MBA, Sutskever, CS, Penn, ECCV, Neural Information Processing Systems, Hugo Larochelle, Courser

PyTorch, Javascript, OpenCV, Tensor, Google Colab, TensorFlow, Tensorflow, PyTorch Library, Torch, Keras, Part 2, GitHub, Keras API, Graph, Jupyter, TensorRT, TensorFlow Object Detection API, GitHub Repo, Repo, Js, Jupyter Notebook, C++, Java, Paperspace, HTML, Browser, Layers, Github, Notebook, Pytorch, React Native, JS, Js Library, JavaScript, Re, CSS, TensorFlow Hub, Fastai, Fastai Library, Caffe, Theano, CNTK, Github Repo, Tensor Flow, GitHub Repository, CoreML, React, D3, Vue, Api, TensorFlows, Caffe2, Quick, Draw, Tensorboard, Git, Tf, DOM, R Python, Django, Object, Vega-Lite, SVG, Github Repository, Colab, RStudio, Vega, Bokeh, Html, D3 API, Gephi, Svg, IPython, Infographs, Infograph, Cookiecutter, Src, Dask, Papermill, Nteract, TensorFlow Lite, Shiny, AutoGraph, PyData, Tensor2Tensor, EFS


Rank:  20 


Basic, W, L, Ft, G, P, X, WRT, B, Ps, Pc, Algorithmic, E, TP, D, BERT, C, Y, O, Attention, Transformer, Emb_dim, Positional, K_i, Q_i, V_i, Et Al, Math, ELMo, The Transformer, H

Yuval Harari, Hindu, Eliezer Yudkowsky, Romans, Nigerian, BlessU-2, God, Syers, God-like, E-, Transhumanists, Buddhist, Greek, Sam Harris, Yuval Noah Harari, Homo Deus, Harari, Ancient Greeks, Minder, Harvards, Jewish, Greeks, Babylonians, Buddha, Gods, Bible, Genesis, Jesus, Renaissance, Enlightenment, Roman, Way, Christianity, Christian, Christians, Eve, Religions, Muslim, Prometheus, Daedalus, Icarus, ISIS, Religion, 20th, Heraclitus, Plato, Sapiens, Socrates, Minos, Poseidon, Cicero, SGSOs


Rank:  45 


EEC, African American, Asian, Bigrams, TF, African-Americans, Holocaust, Jerusalem, CountVectoriser, TfidfTransformer, Asians, Hitler, Cosine, Cosine Similarity, Frequency, Germans, Nazi, TF-IDF, Tf-idf, Whites, Codebook, Na Et Al, NHANES, TFIDF, CountVectorizer, N-grams, Document, Tm, DTM, TDM, Document-term, Vasicek, CIR, RandomForest, Pipeline, BJP, IDF, Additive


Rank:  46 


World War, Second World War, Communism, Israeli, Yemen, Hungary, NSA, Army, Rankism, Civil War, Vladim

In [13]:
clusters[43]

['Dot-com',
 'VC',
 'Data Science',
 'Data',
 'Silicon Valley',
 'UX',
 'McKinsey',
 'RD',
 'Enterprise',
 'WatrHubs',
 'Dell',
 'SaaS',
 'Business Intelligence',
 'BI',
 'McKinsey Global Institute',
 'B2B',
 'Tech',
 'Gartners',
 'Org',
 'PwC',
 'Accenture',
 'Gartner',
 'CRISP-DM',
 'Wells Fargo',
 'CPA',
 'CRM',
 'Salesforce',
 'CPAs',
 'CFOs',
 'Fintech',
 'PayPal',
 'HR',
 'CB Insights',
 'Analytics',
 'HR Department',
 'Excel',
 'BD',
 'IDC',
 'Hire',
 'ABBYs',
 'Capital One',
 'Customer Relationship Management',
 'Startups',
 'Forrester',
 'Big Data',
 'Ruuh',
 'B2C',
 'ERP',
 'FPOs',
 'NPOs',
 'FPO',
 'NPO',
 'AI Council',
 'Marketing',
 'CDO',
 'CAIO',
 'CRO',
 'Information Architecture',
 'Microsoft Excel',
 'Airtable',
 'SCM',
 'Data Scientists',
 'Ethos',
 'Asus']